In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk


In [51]:
df = pd.read_csv('./email_spam.csv',encoding='utf-8')
print(len(df))

84


In [52]:
def tokenizer_helper(max_words,text):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    return tokenizer,sequences

In [53]:
text , labels = np.array(df['text']), np.array(df['type'])

In [54]:
tokenizer,sequences = tokenizer_helper(10000,text=text)

In [55]:
def train_val_split(sentences, labels):
    """
    Splits the dataset into training and validation sets
    
    Args:
        sentences (list of string): lower-cased sentences without stopwords
        labels (list of string): list of labels
        training split (float): proportion of the dataset to convert to include in the train set
    
    Returns:
        train_sentences, validation_sentences, train_labels, validation_labels - lists containing the data splits
    """
    
    ### START CODE HERE
    
    # Compute the number of sentences that will be used for training (should be an integer)
    train_size = int(len(sentences) * 0.8)

    # Split the sentences and labels into train/validation splits
    train_sentences = sentences[:train_size]
    train_labels = labels[:train_size]

    validation_sentences = sentences[train_size:]
    validation_labels = labels[train_size:]
    
    ### END CODE HERE
    
    return train_sentences, validation_sentences, train_labels, validation_labels

In [56]:
train_sentences, val_sentences, train_labels, val_labels = train_val_split(text, labels)

In [57]:
print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_sentences)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 67 sentences for training.

There are 67 labels for training.

There are 17 sentences for validation.

There are 17 labels for validation.


In [58]:
def seq_and_pad(sentences, tokenizer):
    """
    Generates an array of token sequences and pads them to the same length
    
    Args:
        sentences (list of string): list of sentences to tokenize and pad
        tokenizer (object): Tokenizer instance containing the word-index dictionary
        padding (string): type of padding to use
        maxlen (int): maximum length of the token sequence
    
    Returns:
        padded_sequences (array of int): tokenized sentences padded to the same length
    """ 
    
    ### START CODE HERE
       
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    
    # Pad the sequences using the correct padding and maxlen
    padded_sequences = pad_sequences(sequences,maxlen=120,padding='post')
    
    ### END CODE HERE
    
    return padded_sequences

In [59]:
train_padded_seq = seq_and_pad(train_sentences, tokenizer)
val_padded_seq = seq_and_pad(val_sentences, tokenizer)

In [60]:
print(f"Padded training sequences have shape: {train_padded_seq.shape}\n")
print(f"Padded validation sequences have shape: {val_padded_seq.shape}")

Padded training sequences have shape: (67, 120)

Padded validation sequences have shape: (17, 120)


In [61]:
def tokenize_labels(all_labels, split_labels):
    """
    Tokenizes the labels
    
    Args:
        all_labels (list of string): labels to generate the word-index from
        split_labels (list of string): labels to tokenize
    
    Returns:
        label_seq_np (array of int): tokenized labels
    """
    
    ### START CODE HERE
    
    # Instantiate the Tokenizer (no additional arguments needed)
    label_tokenizer = Tokenizer()
    
    # Fit the tokenizer on all the labels
    label_tokenizer.fit_on_texts(split_labels)
    
    # Convert labels to sequences
    label_seq = label_tokenizer.texts_to_sequences(split_labels)
    
    # Convert sequences to a numpy array. Don't forget to substact 1 from every entry in the array!
    label_seq_np = np.array([w[0] - 1 for w in label_seq]).reshape(len(split_labels),1)
    
    ### END CODE HERE
    
    return label_seq_np

In [62]:
train_label_seq = tokenize_labels(labels, train_labels)
val_label_seq = tokenize_labels(labels, val_labels)

print(f"First 5 labels of the training set should look like this:\n{train_label_seq[:5]}\n")
print(f"First 5 labels of the validation set should look like this:\n{val_label_seq[:5]}\n")
print(f"Tokenized labels of the training set have shape: {train_label_seq.shape}\n")
print(f"Tokenized labels of the validation set have shape: {val_label_seq.shape}\n")

First 5 labels of the training set should look like this:
[[0]
 [1]
 [1]
 [1]
 [0]]

First 5 labels of the validation set should look like this:
[[1]
 [1]
 [1]
 [1]
 [1]]

Tokenized labels of the training set have shape: (67, 1)

Tokenized labels of the validation set have shape: (17, 1)



In [63]:
def create_model(num_words, embedding_dim, maxlen):
    """
    Creates a text classifier model
    
    Args:
        num_words (int): size of the vocabulary for the Embedding layer input
        embedding_dim (int): dimensionality of the Embedding layer output
        maxlen (int): length of the input sequences
    
    Returns:
        model (tf.keras Model): the text classifier model
    """
    
    tf.random.set_seed(123)
    
    ### START CODE HERE
    
    model = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 

    ### END CODE HERE

    return model

In [64]:
model = create_model(10000, 10 , 120)

In [65]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 10)           100000    
                                                                 
 flatten_1 (Flatten)         (None, 1200)              0         
                                                                 
 dense_2 (Dense)             (None, 3)                 3603      
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 103,607
Trainable params: 103,607
Non-trainable params: 0
_________________________________________________________________


In [67]:
history = model.fit(train_padded_seq, train_label_seq, epochs=15, validation_data=(val_padded_seq, val_label_seq))

Epoch 1/15
3/3 [==============================] - 0s 82ms/step - loss: 0.5899 - accuracy: 0.7164 - val_loss: 0.6581 - val_accuracy: 0.7059
Epoch 2/15
3/3 [==============================] - 0s 38ms/step - loss: 0.5757 - accuracy: 0.7164 - val_loss: 0.6540 - val_accuracy: 0.7059
Epoch 3/15
3/3 [==============================] - 0s 37ms/step - loss: 0.5624 - accuracy: 0.7164 - val_loss: 0.6501 - val_accuracy: 0.7059
Epoch 4/15
3/3 [==============================] - 0s 37ms/step - loss: 0.5479 - accuracy: 0.7164 - val_loss: 0.6463 - val_accuracy: 0.7059
Epoch 5/15
3/3 [==============================] - 0s 33ms/step - loss: 0.5346 - accuracy: 0.7313 - val_loss: 0.6429 - val_accuracy: 0.7059
Epoch 6/15
3/3 [==============================] - 0s 43ms/step - loss: 0.5202 - accuracy: 0.7313 - val_loss: 0.6398 - val_accuracy: 0.7059
Epoch 7/15
3/3 [==============================] - 0s 42ms/step - loss: 0.5061 - accuracy: 0.7313 - val_loss: 0.6367 - val_accuracy: 0.7059
Epoch 8/15
3/3 [===========

In [69]:
model.save('Spam_classifier.h5')